In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore') #ignore warnings

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv") #load train.csv
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv") #load test.csv
sample = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv") #load sample_submission.csv

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head(10)

In [ ]:
test.head(10)

In [ ]:
sample.head() #type of data need to submit

In [ ]:
for i in [train, test]:
    i.drop('id', axis = 1, inplace = True)

In [ ]:
x = train.drop("loss", axis=1) #feature
y = train["loss"] #traget

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x[x.columns.tolist()] = sc.fit_transform(x[x.columns.tolist()])
test[test.columns.tolist()] = sc.fit_transform(test[test.columns.tolist()])

In [ ]:
lgbm_param = {
        "metric" : "rmse",
        "device_type" : "gpu",
        "boosting_type" : "gbdt",
        "reg_alpha" : 0.4988899107600194,
        "reg_lambda" : 0.3216983283790298,
        "num_leaves" : 64,
        "learning_rate" : 0.03033148284558732,
        "max_depth" : 33,
        "n_estimators" : 5581,
        "min_child_samples" : 77,
        "min_child_weight" : 0.0166605551617851,
        "colsample_bytree" : 0.6207274451420828,
        "subsample" : 0.9566304128710696
    }

In [ ]:
from lightgbm import LGBMRegressor
lightgbm_model = LGBMRegressor(**lgbm_param)

In [ ]:
xgb_param = {
        "eval_metric" : "rmse",
        "random_state" : 228,
        "device_type" : "gpu",
        "booster" : "gbtree",
        "alpha" : 0.00573215966018785,
        "lambda" : 0.00013592165632140884,
        "learning_rate" : 0.010512283852839102,
        "max_depth" : 10,
        "n_estimators" : 2432,
        "min_child_samples" : 30,
        "colsample_bytree" : 0.5825502178882395,
        "tree_method": "gpu_hist"
    }

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(**xgb_param)

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut
from sklearn.ensemble import VotingRegressor

folds = KFold(n_splits = 10, random_state = 228, shuffle = True)
predictions = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(folds.split(x)):
    
    x_train, x_val = x.iloc[trn_idx], x.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    model = VotingRegressor(
            estimators = [
                ('lgbm', lightgbm_model),
                ('xgb', xgb_model)
            ],
            weights = [0.15, 0.7],
            n_jobs = -1
        )
   
    model.fit(x_train, y_train)
    
    predictions += model.predict(test) / folds.n_splits 
    
sample['loss'] = predictions

In [ ]:
sample.to_csv('voteresult.csv', index = False)